# Подготовка обучающего датасета для модели YOLO из видео
## Цель задания: 
**Получить практический опыт в создании базового датасета для детекции объектов в формате YOLO, используя видео продолжительностью 20 секунд и любой удобный инструмент разметки**

## 1.	Выбор видео:
* Выберите видео длительностью около 25 секунд. Видео должно содержать объекты, которые вы планируете детектировать (например, автомобили, люди, животные и т.д.).
*	Убедитесь, что качество видео достаточное для разметки (четкие изображения объектов).

При выборе видео необходимо учитывать наличие на нем четкого изображения объектов, котрые необходимо детектировать. При этом желательно наличие примерно одинакового количества кадров с детектирумыми объектами, а также их изображение с разных ракурсов.

## 2. Извлечение кадров:
* С помощью любого видеоредактора или скрипта извлеките кадры из видео.
* Рекомендуется извлекать кадры с интервалом, чтобы получить примерно 50 изображений.

In [1]:
pip install natsort

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import torch
import cv2
import os
import shutil
import natsort as ns
import json

Для извлечения кадров используем скрипт ниже:

In [18]:
def video2frames(src, out, sample, xr, yr):
    
    if not os.path.exists(out):
        os.mkdir(out)
    
    cap = cv2.VideoCapture(src)
    
    if not cap.isOpened(): 
        print('Ошибка чтения видео-файла')
    
    i, s = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            if i % sample == 0:
                frame = cv2.resize(frame, (xr, yr), cv2.INTER_NEAREST)
                cv2.imwrite(os.path.join(out, os.path.split(src)[-1].split('.')[0] + '_' + str(i) + '.jpg'), frame)
                s += 1
            i += 1
        else:
            break
    cap.release()
        
    return f'Задача завершена. Сохранено кадров: {s}.'

In [19]:
os.path.split(src)[-1].split('.')[0]

'2'

Функция читает видео по па адресу **src** по кадрам и сохраняет их как изображения в директорию **out**. Настраиваем выборку не всех, а каждого i-того кадра (параметр **sample** определяем ниже)

In [20]:
# загрузка видео и определение директорий
name_video = '2.mp4'
folder_video = '01_Задание_1'
src = os.path.join('..', folder_video, name_video)
out = os.path.join(os.curdir, 'frames')
cap = cv2.VideoCapture(src)

# задаем размеры видео
new_x = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
new_y = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# определяем количество кадров видео
frame_count = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# учитывая необходимое количество кадров для разметки,
# равное 50, определяем параметр sample
sample = frame_count // 50

Запускаем скрипт

In [21]:
%%time

video2frames(src, out, sample, new_x, new_y)

Wall time: 16.5 s


'Задача завершена. Сохранено кадров: 56.'

Далее из полученных кадров делаем вырезки 640х640 из собранных кадров, так как именно на 640х640 обучалась YOLO. 

640х640 - оптимальный размер изображений.

Для получения вырезок используем функцию click_sampler.
При запуске функции проиходит открытие стандратного окна OpenCV. Далее с помошью левой кнопки мыши отмечаем центр будущей вырезки на изображении: функция рисует границы вырезки и сохраняет данный фрагмент в указанную директорию (out).
В функции предусмотрен тот момент, что если указать центр вырезки близко к краю изображения, размер рамки остается 640х640 и смещается таким образом, что ее край свомещается с краем изображения.
- Любая кнопка на клавиатуре: вызывает следующее изображение.
- Скрипт завершится после обработки всех изображений!

In [25]:
def click_sampler(name, size, out):
    
    if not os.path.exists(out):
        os.mkdir(out)
        
    if size[0] <= 0 or size[1] <= 0:
        raise ValueError ('tile size shuld be >= 0')
  
    img = cv2.imread(name)
    
    if size[0] > img.shape[0] or size[1] > img.shape[1]:
        print('Warnng: title size > img size')
        pass
    
    img_c = img.copy()

    cv2.namedWindow(name, cv2.WINDOW_NORMAL)
    cv2.imshow(name, img)
    cv2.resizeWindow(name, img.shape[1], img.shape[0])

    coords = []

    def mouse_click(event, x, y, flags, param):
        
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(img, (x, y), 10, (0, 0, 255), -1)
            x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
            xw, yh = x + int(size[0]/2), y + int(size[1]/2)
            cv2.rectangle(img, (x0, y0), (xw, yh), (0, 0, 255), 3)
            cv2.imshow(name, img)
            coords.append((x, y))

        return coords

    cv2.setMouseCallback(name, mouse_click)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    t = 0
    nm, ext = os.path.basename(name.split('.')[0]), os.path.basename(name.split('.')[1])
    
    for coord in coords:
        x, y = coord[0], coord[1]
        x0, y0 = x - int(size[0]/2), y - int(size[1]/2)
        xw, yh = x + int(size[0]/2), y + int(size[1]/2)
        
        if x0 < 0:
            xw = xw - x0
            x0 = 0
        if y0 < 0:
            yh = yh - y0
            y0 = 0
            
        title = img_c[y0:yh, x0:xw]
        print(f"Создан фрейм {os.path.join(out, nm + '_' + str(x) + 'x' + str(y) + '.' + ext)}")
        cv2.imwrite(os.path.join(out, nm + '_' + str(x)+ 'x' + str(y) + '.' + ext), title)
                    
        t += 1
        
    return f'{t} tiles were generated'

In [ ]:
# for series of pic in the folder
pth = 'frames'
out = '640x640'

out_pth = os.path.join(pth, out)

if not os.path.exists(out_pth):
    os.mkdir(out_pth)

for file in ns.natsorted(os.listdir(pth)):
    print(os.path.join(out_pth, os.path.basename(file)))
    click_sampler(os.path.join(pth, os.path.basename(file)), (640,640), out_pth)

## 3. Разметка данных:
* Выберите любой инструмент для разметки данных (например, LabelImg, LabelMe, CVAT, VoTT и т.д.).
* Разметьте объекты на каждом кадре, используя bounding boxes, и присвойте им соответствующие классы.
* Убедитесь, что разметка сохранена в формате, совместимом с YOLO (текстовые файлы с координатами и метками классов).

Для разметки изображений будем использовать VGG Image Annotation (VIA) - браузерный инструмент разметки изображений для создания обучающих выборок.
Инструмент обладает рядом преимуществ, однако для возможности загрузки размеченных изображений в YOLO, буднт необходим скрипт, приведенный ниже.

In [ ]:
def VIA2YOLO_detect(data, cls_codes, out_dir, imgsize):
    
    '''
    Converts VIA detection (x,y,w,h-rects expected) annotations (one .csv file)
    to individual YOLO-formatted txt files for each img with objects.
    Empty imgs have no any files. Params:
    
    -data (path-like string) - source .csv  VIA annotation
    -clscodes (dict) - class name-to-index mapping: {'dog':0, 'cat':1 ...}
    -out - dir for generated .txt files (will be created if not exist)
    -imgsize (tuple of ints) - size (H,W) of annotated imgs (expected all the same)
    '''
    
    annot = pd.read_csv(data, index_col=0)
#     annot.drop(['file_size', 'file_attributes'], axis=1, inplace=True)
    
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
        
    for label, row in annot.iterrows():
        name, r_count, coords, cls = label, int(row[2]), json.loads(row[4]), json.loads(row[5])
        cls = str(cls)
        
#         # with empty files for empty pics 
        if r_count == 0:
            with open(out_dir + str(label[:-4]) + '.txt', 'w') as f:
                f.write('')
        
        if r_count == 1:
            if coords['name'] == 'rect':
                xywh = []
                xywh.append(coords['x']/imgsize[0] + (coords['width']/imgsize[0])/2)
                xywh.append(coords['y']/imgsize[1] + (coords['height']/imgsize[1])/2)
                xywh.append(coords['width']/imgsize[0])
                xywh.append(coords['height']/imgsize[1])

                for key in cls_codes.keys():
                    if key in cls:
                        cls_id = cls_codes[key]

                fin_str = str(cls_id) + ' ' + str(xywh)[1:-1].replace(',','')

                with open(out_dir + str(label[:-4]) + '.txt', 'w') as f:
                    f.write(fin_str + '\n')
            
        if r_count > 1:
            if coords['name'] == 'rect':
                xywh = []
                xywh.append(coords['x']/imgsize[0] + (coords['width']/imgsize[0])/2)
                xywh.append(coords['y']/imgsize[1] + (coords['height']/imgsize[1])/2)
                xywh.append(coords['width']/imgsize[0])
                xywh.append(coords['height']/imgsize[1])

                for key in cls_codes.keys():
                    if key in cls:
                        cls_id = cls_codes[key]

                fin_str = str(cls_id) + ' ' + str(xywh)[1:-1].replace(',','')

                with open(out_dir + str(label[:-4]) + '.txt', 'a') as f:
                    f.write(fin_str + '\n')
    
    return annot['region_attributes'].value_counts()

In [ ]:
data = 'from_VIA_detect.csv'
out_dir = './data_labels/'

# словарь имен классов - задает соответствие между текстовой разметкой из VIA и индексами для формата YOLO
mcodes = { 
    'object_A': 0,
    'object_B': 1,
    'object_C': 2,
}

mcodes = {'bear': 0} # единственный класс

VIA2YOLO_detect(data, mcodes, out_dir, (640, 640))

## 4. Структура датасета:
* Создайте папки для изображений и аннотаций (например, images/ и labels/).
* Проверьте, что имена файлов изображений и соответствующих аннотаций совпадают (например, image1.jpg и image1.txt).

## 5. Проверка данных:
* Выберите несколько изображений и убедитесь, что bounding boxes отображаются корректно.
* Проверьте формат аннотаций на соответствие требованиям YOLO.

## 6. Подготовка отчета:
* Напишите краткий отчет (1-2 страницы), в котором опишите:
* Процесс выбора видео и извлечения кадров.
* Используемый инструмент для разметки и опыт его использования.
* Возможные сложности и способы их решения.
* Приложите несколько примеров размеченных изображений.
